# Project based Data Analyst: Skills test

The type of data you’ll be working with originates from projects all over the world, and therefore comes in various languages. To assess your data cleaning skills, we have provided a .csv of geotechnical data from a previous project in Brazil, which needs to be set to the ‘EMerald standard’. 

EMerald’s internal data structures are built around the Swedish geotechnical format – we have an open source library called libsgfdata (installable via pip or by cloning the github repo and following the README.me.

You can see an excel sheet named SP_TRO-TEL_88.xlsx in the folder. We have also provided a spreadsheet and the corresponding parsed sgf file SP_TRO-TEL_94.sgf / .xlsx. As you can see this SGF file does not contain all the data from the excel sheet, but only the subset that we’re interested in.

Your task is to extract a similar dataset, or as much data as you can out of SP_TRO-TEL_88.xlsx and write an SGF file using the method outlined in the libsgfdata documentation. Please use the jupyter notebook (libsgfdata_file_reader.ipynb) provided and leave all code used available so that your work is replicable.

We understand the data is in Portuguese, so we expect that the key libsgfdata columns are translated (use google translate!) 

Good luck!


# Spare link to libsgfdata repo: 

https://github.com/emerald-geomodelling/libsgfdata with documentation on how to write an SGF file

In [7]:
import libsgfdata

In [8]:
sgf_file = libsgfdata.SGFData("SP_TRO-TEL_94.sgf")

In [9]:
sgf_file.main

,CONTRATANTE,OBSERVAÇĂO,LOCAL,CIDADE,FISCAL,INÍCIO LAVAGEM,COMP. REVESTIMENTO,Título,method_code,x_coordinate,y_coordinate,work_or_project_number,section,investigation_point,date,responsible_drilling_operator_or_measurement_operator,project_name
0,Subcontratada Enefer,Profundidade de projeto 15m;Năo atingiu a prof...,MT - MATO GROSSO,Rondonópolis,Ediemes Pereira Neves,2.45,"1,50",Perfil de Sondagem SPT,standard_penetration_test,"E 737.711,456","N 8.200.240,215",SP_TRO-TEL_94,SP_TRO-TEL_94,SP - 94,2020-11-17,Luiz Carlos da Conceiçăo,Projeto Básico - Ferrovia Lucas do Rio Verde -...


In [10]:
sgf_file.data

,depth,test_no,ramming_nn_value,ramming_nn_1_value,comments,end_depth,start_depth,investigation_point
0,-1.00,0.0,0,0,Areia siltosa cinza,-1.00,0.00,SP - 94
1,-1.45,1.0,4,4,Areia siltosa cinza,-1.45,-1.00,SP - 94
2,-2.45,2.0,7,8,Areia siltosa cinza,-2.45,-1.45,SP - 94
3,-3.45,3.0,8,8,Areia siltosa cinza,-3.45,-2.45,SP - 94
4,-4.45,4.0,12,14,Areia siltosa cinza;v,-4.45,-3.45,SP - 94
5,-5.45,5.0,16,17,Areia siltosa cinza,-5.45,-4.45,SP - 94
6,-6.45,6.0,12,11,Areia siltosa cinza,-6.45,-5.45,SP - 94
7,-7.45,7.0,13,13,Areia siltosa cinza,-7.45,-6.45,SP - 94
8,-8.45,8.0,17,18,Areia siltosa cinza,-8.45,-7.45,SP - 94
9,-9.45,9.0,21,24,Areia siltosa cinza,-9.45,-8.45,SP - 94


Please put all working code below here

# ---------------------------------------

In [32]:
import pandas as pd
import libsgfdata
import glob
import os

In [42]:


def parse_files(in_folder,out_folder):
    
    def get_main(df):
        main_data="$\n"
        fields=["CONTRATANTE=","LOCAL=","CIDADE=","FISCAL=","COMP. REVESTIMENTO=","TITULO=","HX=","HY=","HJ=","HL=","HK=","KD=","Person=","KP="]
        coords=[df.loc[44,1],df.loc[48,1],df.loc[50,1],df.loc[49,16],df.loc[52,8],df.loc[3,0][8:len(df.loc[3,0])],df.loc[39,16],df.loc[40,16],df.loc[5,0][8:len(df.loc[5,0])],df.loc[4,15][9:len(df.loc[4,15])],df.loc[5,16],df.loc[46,16],df.loc[50,16],df.loc[46,1]]
        for i in range(len(coords)):
            if str(coords[i])!="":
                main_data+=fields[i]+str(coords[i])+","
            else:
                pass
        return main_data[:-1]+"\n"

    def get_data(df):
        data="#\n"
        fields=["D=","ProvNr=","Nn=","Nn+1=","K=","AO=","AN="]
        columns=[0,4,6,7,14,0]
        for row in range(13,35):
            if str(df.loc[row,0])!="":
                line=""
                for i in range(len(columns)):
                    line+=fields[i]+str(df.loc[row,columns[i]])+","
                if row==13:
                    line+=fields[6]+"0\n"
                else:
                    line+=fields[6]+str(df.loc[row-1,0])+"\n"
                data+=line       
            else:
                return data[:-1]
    
    for file in glob.glob(os.path.join(in_folder,"*.xlsx")):
        df=pd.read_excel(file,sheet_name="Sondagem",header=None,names=[i for i in range(31)]).fillna("")
        new_file = file[5:-5]+".sgf"
        with open(out_folder+new_file,"w") as document:
            document.write(get_main(df)+get_data(df))
        libsgfdata.SGFData(out_folder+new_file).dump
        


In [34]:
#TEST

In [35]:
in_route="data/"
out_route="done/"
parse_files(in_route,out_route)

In [36]:
test1=libsgfdata.SGFData("done/SP_TRO-TEL_88.sgf")

In [37]:
test1.main

,CONTRATANTE,LOCAL,CIDADE,FISCAL,TITULO,x_coordinate,y_coordinate,work_or_project_number,section,investigation_point,date,responsible_drilling_operator_or_measurement_operator,project_name
0,Subcontratada Enefer,MT - MATO GROSSO,Rondonópolis,Ediemes Pereira Neves,Perfil de Sondagem SPT,"E 739.017,126","N 8.195.326,187",SP_TRO-TEL_88,SP_TRO-TEL_88,SP - 88,2020-11-12,Rodrigo Borges Leonel,Projeto Básico - Ferrovia Lucas do Rio Verde -...


In [38]:
test1.data

,depth,test_no,ramming_nn_value,ramming_nn_1_value,comments,end_depth,start_depth,investigation_point
0,-1.00,0.0,0,0,Areia siltosa amarela,-1.00,0.00,SP - 88
1,-1.45,1.0,4,4,Areia siltosa amarela,-1.45,-1.00,SP - 88
2,-2.45,2.0,4,3,Areia siltosa amarela,-2.45,-1.45,SP - 88
3,-3.45,3.0,4,5,Areia siltosa amarela,-3.45,-2.45,SP - 88
4,-4.45,4.0,6,7,Areia amarela,-4.45,-3.45,SP - 88
5,-5.45,5.0,6,5,Areia amarela,-5.45,-4.45,SP - 88
6,-6.45,6.0,10,11,Areia amarela,-6.45,-5.45,SP - 88
7,-7.45,7.0,10,11,Areia amarela,-7.45,-6.45,SP - 88
8,-8.45,8.0,13,15,Areia amarela,-8.45,-7.45,SP - 88
9,-9.45,9.0,16,15,Areia amarela,-9.45,-8.45,SP - 88


In [39]:
test2=libsgfdata.SGFData("done/SP_TRO-TEL_94.sgf")

In [40]:
test2.main

,CONTRATANTE,LOCAL,CIDADE,FISCAL,COMP. REVESTIMENTO,TITULO,x_coordinate,y_coordinate,work_or_project_number,section,investigation_point,date,responsible_drilling_operator_or_measurement_operator,project_name
0,Subcontratada Enefer,MT - MATO GROSSO,Rondonópolis,Ediemes Pereira Neves,"1,50",Perfil de Sondagem SPT,"E 737.711,456","N 8.200.240,215",SP_TRO-TEL_94,SP_TRO-TEL_94,SP - 94,2020-11-17,Luiz Carlos da Conceição,Projeto Básico - Ferrovia Lucas do Rio Verde -...


In [41]:
test2.data

,depth,test_no,ramming_nn_value,ramming_nn_1_value,comments,end_depth,start_depth,investigation_point
0,-1.00,0.0,0,0,Areia siltosa cinza,-1.00,0.00,SP - 94
1,-1.45,1.0,4,4,Areia siltosa cinza,-1.45,-1.00,SP - 94
2,-2.45,2.0,7,8,Areia siltosa cinza,-2.45,-1.45,SP - 94
3,-3.45,3.0,8,8,Areia siltosa cinza,-3.45,-2.45,SP - 94
4,-4.45,4.0,12,14,Areia siltosa cinza,-4.45,-3.45,SP - 94
5,-5.45,5.0,16,17,Areia siltosa cinza,-5.45,-4.45,SP - 94
6,-6.45,6.0,12,11,Areia siltosa cinza,-6.45,-5.45,SP - 94
7,-7.45,7.0,13,13,Areia siltosa cinza,-7.45,-6.45,SP - 94
8,-8.45,8.0,17,18,Areia siltosa cinza,-8.45,-7.45,SP - 94
9,-9.45,9.0,21,24,Areia siltosa cinza,-9.45,-8.45,SP - 94
